# Data Imports

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

pd.set_option('display.max_columns', 70) # Set max display to 70 columns

## Importing the 2020 air quality CSV from the [EPA website](https://aqs.epa.gov/aqsweb/airdata/download_files.html)  

https://aqs.epa.gov/aqsweb/airdata/download_files.html

In [2]:
air_quality_df = pd.read_csv("annual_conc_by_monitor_2020.csv")

In [3]:
air_quality_df = air_quality_df[air_quality_df['State Code'] == 49]

In [4]:
air_quality_df.head()

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Metric Used,Method Name,Year,Units of Measure,Event Type,Observation Count,Observation Percent,Completeness Indicator,Valid Day Count,Required Day Count,Exceptional Data Count,Null Data Count,Primary Exceedance Count,Secondary Exceedance Count,Certification Indicator,Num Obs Below MDL,Arithmetic Mean,Arithmetic Standard Dev,1st Max Value,1st Max DateTime,2nd Max Value,2nd Max DateTime,3rd Max Value,3rd Max DateTime,4th Max Value,4th Max DateTime,1st Max Non Overlapping Value,1st NO Max DateTime,2nd Max Non Overlapping Value,2nd NO Max DateTime,99th Percentile,98th Percentile,95th Percentile,90th Percentile,75th Percentile,50th Percentile,10th Percentile,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
66484,49,3,7001,42601,1,41.945874,-112.233973,WGS84,Nitric oxide (NO),1 HOUR,NaN,Observed Values,Instrumental - Chemiluminescence Teledyne API ...,2020,Parts per billion,No Events,8123,92,Y,355,366,0,661,NaN,NaN,Certification not required,0,0.125200,0.506018,8.000,2020-01-21 14:00,7.000,2020-01-21 13:00,7.000,2020-12-09 13:00,7.000,2020-12-09 14:00,NaN,NaN,NaN,NaN,2.000,2.000,1.000,0.00,0.000,0.000,0.000,WASHAKIE,"8600 WEST 24000 NORTH PORTAGE, UTAH",Utah,Box Elder,NaN,"Ogden-Clearfield, UT",2021-02-09
66485,49,3,7001,42602,1,41.945874,-112.233973,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour 2010,Daily Maximum 1-hour average,Instrumental - Chemiluminescence Teledyne API ...,2020,Parts per billion,No Events,8124,92,Y,355,366,0,660,0.0,NaN,Certification not required,0,2.247887,2.545732,16.000,2020-12-09 15:00,14.000,2020-01-21 14:00,14.000,2020-12-10 18:00,12.000,2020-01-01 06:00,NaN,NaN,NaN,NaN,12.000,11.000,7.000,6.00,3.000,2.000,0.000,WASHAKIE,"8600 WEST 24000 NORTH PORTAGE, UTAH",Utah,Box Elder,NaN,"Ogden-Clearfield, UT",2024-05-22
66486,49,3,7001,42602,1,41.945874,-112.233973,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 Annual 1971,Observed values,Instrumental - Chemiluminescence Teledyne API ...,2020,Parts per billion,No Events,8124,92,Y,355,366,0,660,NaN,NaN,Certification not required,0,0.828040,1.483330,16.000,2020-12-09 15:00,16.000,2020-12-09 16:00,16.000,2020-12-09 17:00,16.000,2020-12-09 18:00,NaN,NaN,NaN,NaN,7.000,5.000,4.000,2.00,1.000,0.000,0.000,WASHAKIE,"8600 WEST 24000 NORTH PORTAGE, UTAH",Utah,Box Elder,NaN,"Ogden-Clearfield, UT",2024-05-22
66487,49,3,7001,42603,1,41.945874,-112.233973,WGS84,Oxides of nitrogen (NOx),1 HOUR,NaN,Observed Values,Instrumental - Chemiluminescence Teledyne API ...,2020,Parts per billion,No Events,8124,92,Y,355,366,0,660,NaN,NaN,Certification not required,0,1.410758,1.873058,23.000,2020-01-21 14:00,22.000,2020-12-09 14:00,20.000,2020-01-21 13:00,20.000,2020-12-09 15:00,NaN,NaN,NaN,NaN,9.000,7.000,5.000,3.00,2.000,1.000,0.000,WASHAKIE,"8600 WEST 24000 NORTH PORTAGE, UTAH",Utah,Box Elder,NaN,"Ogden-Clearfield, UT",2021-02-09
66488,49,3,7001,44201,1,41.945874,-112.233973,WGS84,Ozone,1 HOUR,Ozone 1-hour 1979,Daily maxima of observed hourly values (betwee...,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,2020,Parts per million,No Events,7552,90,Y,330,366,0,1232,0.0,0.0,Certification not required,0,0.048018,0.009216,0.093,2020-08-21 14:00,0.092,2020-08-22 14:00,0.075,2020-08-23 11:00,0.069,2020-07-09 15:00,NaN,NaN,NaN,NaN,0.069,0.069,0.063,0.06,0.054,0.047,0.037,WASHAKIE,"8600 WEST 24000 NORTH PORTAGE, UTAH",Utah,Box Elder,NaN,"Ogden-Clearfield, UT",2024-05-27


## Importing the shape file

The shape files can be downloaded from: 
https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html#list-tab-790442341  

We only added the relevant shape file to our GitHub as it would be too large with everything in it. We are using these files:  
* tl_2020_49_state20.shp
* tl_2020_49_state20.shx
* tl_2020_49_state20.dbf
* tl_2020_49_state20.prj

In [5]:
# The shape file is in the 'tl_2020_49_all' folder -> tl_2020_49_state20.shp
# If you want to run, you'll have to replace with your own file path
file_path = "C:/Users/joshu/Documents/GitHub/BMI6016_SDOH/tl_2020_49_all/tl_2020_49_state20.shp"
census_tracts = gpd.read_file(file_path)

In [6]:
census_tracts.head()

,REGION20,DIVISION20,STATEFP20,STATENS20,GEOID20,STUSPS20,NAME20,LSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,4,8,49,01455989,49,UT,Utah,00,G4000,A,213355058738,6529987297,+39.3349925,-111.6563326,"POLYGON ((-114.04703 39.9061, -114.04702 39.90..."


In [7]:
print(census_tracts.crs)
# Convert the census_tract GeoDF from EPSG:4269 (NAD83) to EPSG:4326 (WGS 84)
# This will match the format of the geo_df
census_tracts = census_tracts.to_crs(epsg=4326) 
print(census_tracts.crs)

EPSG:4269
EPSG:4326


In [8]:
# Creating point geometries from latitude and longitude

geometry = [Point(xy) for xy in zip(air_quality_df['Longitude'], air_quality_df['Latitude'])]
geo_df = gpd.GeoDataFrame(air_quality_df, geometry=geometry, crs="EPSG:4326")  # Assuming your data is in WGS 84 (EPSG:4326)

In [9]:
# Spatial join between the point GeoDF and census tract DF
joined_df = gpd.sjoin(geo_df, census_tracts, predicate='within', how='left') #using left join to keep all original data.

In [10]:
# Extract GEOID from joined GeoDF
if 'GEOID' in joined_df.columns:
    result_df = joined_df[['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC', 'Latitude', 'Longitude', 'GEOID']]
elif 'GEOID20' in joined_df.columns:
    result_df = joined_df[['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC', 'Latitude', 'Longitude', 'GEOID20']]
else:
    result_df = joined_df[['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC', 'Latitude', 'Longitude']] # if no GEOID information, keep original data.
    print("Warning: No GEOID or GEOID20 column found in census tract data.")

In [11]:
joined_df.head()

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Metric Used,Method Name,Year,Units of Measure,Event Type,Observation Count,Observation Percent,Completeness Indicator,Valid Day Count,Required Day Count,Exceptional Data Count,Null Data Count,Primary Exceedance Count,Secondary Exceedance Count,Certification Indicator,Num Obs Below MDL,Arithmetic Mean,Arithmetic Standard Dev,1st Max Value,1st Max DateTime,2nd Max Value,2nd Max DateTime,3rd Max Value,3rd Max DateTime,...,4th Max DateTime,1st Max Non Overlapping Value,1st NO Max DateTime,2nd Max Non Overlapping Value,2nd NO Max DateTime,99th Percentile,98th Percentile,95th Percentile,90th Percentile,75th Percentile,50th Percentile,10th Percentile,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,geometry,index_right,REGION20,DIVISION20,STATEFP20,STATENS20,GEOID20,STUSPS20,NAME20,LSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20
66484,49,3,7001,42601,1,41.945874,-112.233973,WGS84,Nitric oxide (NO),1 HOUR,NaN,Observed Values,Instrumental - Chemiluminescence Teledyne API ...,2020,Parts per billion,No Events,8123,92,Y,355,366,0,661,NaN,NaN,Certification not required,0,0.125200,0.506018,8.000,2020-01-21 14:00,7.000,2020-01-21 13:00,7.000,2020-12-09 13:00,...,2020-12-09 14:00,NaN,NaN,NaN,NaN,2.000,2.000,1.000,0.00,0.000,0.000,0.000,WASHAKIE,"8600 WEST 24000 NORTH PORTAGE, UTAH",Utah,Box Elder,NaN,"Ogden-Clearfield, UT",2021-02-09,POINT (-112.23397 41.94587),0,4,8,49,01455989,49,UT,Utah,00,G4000,A,213355058738,6529987297,+39.3349925,-111.6563326
66485,49,3,7001,42602,1,41.945874,-112.233973,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 1-hour 2010,Daily Maximum 1-hour average,Instrumental - Chemiluminescence Teledyne API ...,2020,Parts per billion,No Events,8124,92,Y,355,366,0,660,0.0,NaN,Certification not required,0,2.247887,2.545732,16.000,2020-12-09 15:00,14.000,2020-01-21 14:00,14.000,2020-12-10 18:00,...,2020-01-01 06:00,NaN,NaN,NaN,NaN,12.000,11.000,7.000,6.00,3.000,2.000,0.000,WASHAKIE,"8600 WEST 24000 NORTH PORTAGE, UTAH",Utah,Box Elder,NaN,"Ogden-Clearfield, UT",2024-05-22,POINT (-112.23397 41.94587),0,4,8,49,01455989,49,UT,Utah,00,G4000,A,213355058738,6529987297,+39.3349925,-111.6563326
66486,49,3,7001,42602,1,41.945874,-112.233973,WGS84,Nitrogen dioxide (NO2),1 HOUR,NO2 Annual 1971,Observed values,Instrumental - Chemiluminescence Teledyne API ...,2020,Parts per billion,No Events,8124,92,Y,355,366,0,660,NaN,NaN,Certification not required,0,0.828040,1.483330,16.000,2020-12-09 15:00,16.000,2020-12-09 16:00,16.000,2020-12-09 17:00,...,2020-12-09 18:00,NaN,NaN,NaN,NaN,7.000,5.000,4.000,2.00,1.000,0.000,0.000,WASHAKIE,"8600 WEST 24000 NORTH PORTAGE, UTAH",Utah,Box Elder,NaN,"Ogden-Clearfield, UT",2024-05-22,POINT (-112.23397 41.94587),0,4,8,49,01455989,49,UT,Utah,00,G4000,A,213355058738,6529987297,+39.3349925,-111.6563326
66487,49,3,7001,42603,1,41.945874,-112.233973,WGS84,Oxides of nitrogen (NOx),1 HOUR,NaN,Observed Values,Instrumental - Chemiluminescence Teledyne API ...,2020,Parts per billion,No Events,8124,92,Y,355,366,0,660,NaN,NaN,Certification not required,0,1.410758,1.873058,23.000,2020-01-21 14:00,22.000,2020-12-09 14:00,20.000,2020-01-21 13:00,...,2020-12-09 15:00,NaN,NaN,NaN,NaN,9.000,7.000,5.000,3.00,2.000,1.000,0.000,WASHAKIE,"8600 WEST 24000 NORTH PORTAGE, UTAH",Utah,Box Elder,NaN,"Ogden-Clearfield, UT",2021-02-09,POINT (-112.23397 41.94587),0,4,8,49,01455989,49,UT,Utah,00,G4000,A,213355058738,6529987297,+39.3349925,-111.6563326
66488,49,3,7001,44201,1,41.945874,-112.233973,WGS84,Ozone,1 HOUR,Ozone 1-hour 1979,Daily maxima of observed hourly values (betwee...,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,2020,Parts per million,No Events,7552,90,Y,330,366,0,1232,0.0,0.0,Certification not required,0,0.048018,0.009216,0.093,2020-08-21 14:00,0.092,2020-08-22 14:00,0.075,2020-08-23 11:00,...,2020-07-09 15:00,NaN,NaN,NaN,NaN,0.069,0.069,0.063,0.06,0.054,0.047,0

In [12]:
# saving results
#result_df.to_csv("census_tract_results.csv", index=False)